In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW vw_flattened_mapping AS

-- PART 1: Primary Assessable Unit (Columns B, C, D)
SELECT 
    TRIM(CAST(CostCenter AS STRING)) AS Raw_CC,
    TRIM(AssessableUnitID) AS AU_ID,          
    TRIM(AssessableUnitName) AS AU_Name,      
    TRIM(Segment) AS Segment                             
FROM hive_metastore.ra_adido_2025.fy25_cost_center_mapping
WHERE AssessableUnitID IS NOT NULL

UNION ALL

-- PART 2: Additional Assessable Unit
SELECT 
    TRIM(CAST(CostCenter AS STRING)) AS Raw_CC,
    TRIM(AdditionalAUID) AS AU_ID,  -- Always Col F
    
    -- AU Name Logic (Checking Col E)
    CASE 
        WHEN TRIM(AdditionalAssessableUnitIDandNameandSegment) = 'Yes' 
        THEN TRIM(AssessableUnitName)
        ELSE TRIM(SPLIT(AdditionalAssessableUnitIDandNameandSegment, '-')[1]) 
    END AS AU_Name,
    
    -- Segment Logic (Checking Col E)
    CASE 
        WHEN TRIM(AdditionalAssessableUnitIDandNameandSegment) = 'Yes' 
        THEN TRIM(Segment)
        ELSE TRIM(SPLIT(AdditionalAssessableUnitIDandNameandSegment, '-')[2]) 
    END AS Segment

FROM hive_metastore.ra_adido_2025.fy25_cost_center_mapping
WHERE AdditionalAUID IS NOT NULL AND TRIM(AdditionalAUID) != '';